In [1]:
from numpy import *
from scipy.sparse import *
from sklearn.preprocessing import *

In [21]:
#IDFtracks = load_npz("target_tracks_tags_TF_IDF.npz")
IDFtracks = load_npz("reducedTargetT.npz")
IDFplaylists = load_npz("target_playlist_tags_TF_IDF.npz")

In [22]:
IDFplaylists = normalize(IDFplaylists)

In [23]:
IDFsimil = IDFplaylists * IDFtracks.T

In [24]:
PlwithArtist = load_npz("targetPlaylistArtistReduced.npz")
PlwithArtist = normalize(PlwithArtist, norm="max")
tTracksWithArtist = load_npz("targetTracksArtistReduced.npz")

In [25]:
artistiSimil = PlwithArtist * tTracksWithArtist.T

In [26]:
artistiSimil

<10000x32195 sparse matrix of type '<class 'numpy.float64'>'
	with 3080434 stored elements in Compressed Sparse Row format>

In [27]:
finalSimil = IDFsimil + artistiSimil

In [28]:
monoArtist=load("target_playlist_mono_artist.npy")
monoArtistPlID=monoArtist[:,0]
monoArtistArtistID=monoArtist[:,1]
artistIndexes=load("uniqueArtists_NeededToIndexThe_ArtistReducedMatrices.npy")
artistTracks=load_npz("artists_with_tracksID_ordered_by_occurrencies.npz")
target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
playlists_with_tracks=load("playlists_with_tracks.npy")
target_tracks_ordered = load("targetTracksOrdered.npy")
def getsimil(pls, similrow, n):
    maxi = flip(argsort(similrow), axis=0)
    r = []
    for m in maxi:
        if(not isin(target_tracks_ordered[m], pls[1:])):
           r.append(target_tracks_ordered[m])
           if(len(r)==n):
               return r

In [30]:
fname = "TFIDF_mono_TagSimilarity.csv"
with open(fname,"a") as myfile:
    myfile.write("playlist_id,track_ids\n")
    for pl, similPT, similsum in zip(playlists_with_tracks, IDFsimil, finalSimil):
        plID=pl[0]
        s = str(int(plID))
        s += ","
        if(isin(plID,monoArtistPlID)):#guarda se c'è un artista solo
            artist=monoArtistArtistID[where(monoArtistPlID==plID)[0][0]]#l'unico artista della playlist
            artistIndex=where(artistIndexes==artist)[0][0]#cerca la sua posizione nella matrice delle tracce
            thisArtistTracks=artistTracks.getrow(artistIndex).data#prende le sue tracce
            rr=array([t for t in thisArtistTracks if t not in pl[1:] and isin(t, target_tracks_ordered)])#prende tutte quelle non già presenti
            
            if(len(rr)>=5):
                r = array(rr).take(range(5))
            else:
                r4 = getsimil(pl, ravel(similsum.todense()), 5)
                r4 = array([el for el in r4 if el not in rr])
                r = concatenate((rr, r4.take(range(5-len(rr)))))
        else:
            r = getsimil(pl, ravel(similsum.todense()), 5)#non c'è un artista solo quindi guarda solo i tag
        for el in r:
            s+=str(el)
            s+=" "
        myfile.write(s + "\n")